In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/keyphareextract/NerData.json


In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == "GPU"]

print(get_available_gpus())

2025-05-27 09:38:43.423968: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748338723.918418      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748338724.049245      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


['/device:GPU:0', '/device:GPU:1']


I0000 00:00:1748338743.396010      19 gpu_device.cc:2022] Created device /device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1748338743.396820      19 gpu_device.cc:2022] Created device /device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


## 1. Load Data

In [4]:
import json


with open("/kaggle/input/keyphareextract/NerData.json","r",encoding="utf-8") as f:
    data = json.load(f)

## 2. Gán Nhãn

### 2.1 Gán nhãn thử 1 câu

In [5]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

text = data[2]["text"]
label = data[2]["label"]


tokens = tokenizer(text, truncation=True, return_tensors="pt", padding=True, max_length=256)
tokens_list = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])
# print(len(tokens_list))

tokens_list = [token.replace("@@", "") for token in tokens_list]
tokens_label = ["O"] * len(tokens_list)

print(tokens_list)

special_tokens = ["<s>", "</s>", "<pad>", "<unk>", "<mask>"]

offsets = []
current_pos = 0


# Tìm vị trí của các token trong văn bản gốc
for token in tokens_list:
    if token in special_tokens:
        offsets.append((0, 0))
        continue
    start_pos = text.find(token, current_pos)
    end_pos = start_pos + len(token)
    
    current_pos = end_pos
    
    offsets.append((start_pos, end_pos))

print(offsets)
# Đánh nhãn các token với nhãn tương ứng
for i,token in enumerate(tokens_list):
    if token in special_tokens:
        tokens_label[i] = -100
        continue
    
    for item in label:
        if offsets[i][0] >=item["start"] and offsets[i][0] < item["end"]:
            tokens_label[i] = item["labels"][0]
print(tokens_label)
        

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

['<s>', '<unk>', 'xe', 'Minh', 'Ohio', 'Mới', 'mua', 'bị', 'đập', 'Tan', 'nát', 'ở', 'Hous', 'ton', '<unk>', '!', 'Nên', 'đổi', 'xe', 'Vin', 'f', 'ast', '?', '</s>']
[(0, 0), (0, 0), (2, 4), (5, 9), (10, 14), (15, 18), (19, 22), (23, 25), (26, 29), (30, 33), (34, 37), (38, 39), (40, 44), (44, 47), (0, 0), (47, 48), (50, 53), (54, 57), (58, 60), (61, 64), (64, 65), (65, 68), (68, 69), (0, 0)]
[-100, -100, 'B-ENTITY', 'I-ENTITY', 'I-ENTITY', 'I-ENTITY', 'I-ENTITY', 'I-ENTITY', 'I-ENTITY', 'I-ENTITY', 'I-ENTITY', 'I-ENTITY', 'I-ENTITY', 'I-ENTITY', -100, 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', -100]


### 2.2 Tạo hàm tokenize_and_algn_labels để gán nhãn cho bộ data

In [6]:
def tokenize_and_align_labels(data, tokenizer):
    # for data in data:
    special_tokens = ["<s>", "</s>", "<pad>", "<unk>", "<mask>"]
    text = data["text"]
    label = data["label"]

    tokens = tokenizer(text, truncation=True, return_tensors="pt", padding=True,max_length=256)
    tokens_list = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])
    tokens_list = [token.replace("@@", "") for token in tokens_list]
    tokens_label = ["O"] * len(tokens_list)

    Input_ids = []
    Tokens_label = []
    offsets = []
    current_pos = 0

    # Tìm vị trí của các token trong văn bản gốc
    for token in tokens_list:
        if token in special_tokens:
            offsets.append((0, 0))
            continue
        start_pos = text.find(token, current_pos)
        end_pos = start_pos + len(token)
        current_pos = end_pos
        offsets.append((start_pos, end_pos))

    # Đánh nhãn các token với nhãn tương ứng
    for i, token in enumerate(tokens_list):
        if token in special_tokens:
            tokens_label[i] = -100
            continue
        for item in label:
            if offsets[i][0] >= item["start"] and offsets[i][0] < item["end"]:
                tokens_label[i] = item["labels"][0]

        # Input_ids.append(tokens["input_ids"][0])
        # Tokens_label.append(tokens_label)
    return tokens["input_ids"][0], tokens_label

### 2.3 Thực hiện gán cho toàn bộ data

In [7]:
Input_ids = []
Tokens_label = []



for item in data[:4900]:
    input_ids, tokens_label= tokenize_and_align_labels(item, tokenizer)
    Input_ids.append(input_ids)
    Tokens_label.append(tokens_label)

print(len(Input_ids))

4899


### 2.4 Chuyển nhãn thành số tương ứng trước khi đưa vào dataset

In [8]:
unique_labels = ["O", "B-ENTITY", "I-ENTITY"]
id2label = {i: label for i, label in enumerate(unique_labels)}
label2id = {label: i for i, label in enumerate(unique_labels)}


def convert_labels_to_id(Tokens_label, label2id):
    for item in Tokens_label:
        for i, label in enumerate(item):
            if label in unique_labels:
                item[i] = label2id[label]
            else:
                item[i] = -100
    return Tokens_label


Tokens_label_id = convert_labels_to_id(Tokens_label, label2id)
print(Tokens_label_id[0])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [9]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
# Chuyển sang Dataset

dataset = Dataset.from_dict({"input_ids": Input_ids, "labels": Tokens_label_id})
# Chia tập dữ liệu thành train và test
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

In [10]:
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 3919
})
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 980
})


In [11]:
print(test_dataset[0])

{'input_ids': [0, 3, 3, 3, 870, 43263, 768, 44007, 968, 1042, 9179, 1345, 43410, 13386, 31811, 2275, 2576, 1493, 1529, 3, 3, 3, 3, 30439, 1600, 14786, 27, 1396, 3, 3, 6993, 193, 28, 5321, 22592, 1031, 3365, 26839, 236, 21091, 1387, 1677, 3322, 381, 2073, 40, 119, 1396, 64, 133, 1611, 9244, 119, 1096, 182, 64, 567, 525, 54, 11685, 48, 1456, 2008, 1883, 5, 3, 1584, 60268, 12495, 1493, 95, 24350, 20330, 3, 3, 9990, 6169, 6454, 6302, 36314, 8, 16, 61, 13884, 437, 3518, 1659, 16907, 1384, 4, 47, 13, 2908, 653, 458, 1157, 10, 1602, 626, 2023, 1325, 5516, 4241, 33, 602, 11, 381, 3960, 11, 13, 207, 3153, 57246, 1384, 4, 10, 4623, 47, 1087, 8109, 2008, 12584, 3, 13834, 3884, 543, 2288, 5747, 27, 3, 3, 7852, 3056, 23228, 11083, 55992, 1850, 4, 418, 3139, 3, 7852, 58, 1995, 38, 10, 249, 2492, 1169, 2023, 1325, 3, 2590, 1157, 37194, 4, 1169, 5344, 3, 31743, 1157, 4650, 203, 3, 2590, 1157, 1852, 1565, 515, 1339, 348, 1169, 28, 1565, 574, 1228, 2008, 12584, 2], 'labels': [-100, -100, -100, -100, 1, 

## 3. Tạo Model

In [12]:
from transformers import AutoModelForTokenClassification

unique_labels = ["O","B-ENTITY","I-ENTITY"]
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {i: label for label, i in label_to_id.items()}

model = AutoModelForTokenClassification.from_pretrained(
    "vinai/phobert-base",
    num_labels=len(unique_labels),
    label2id=label_to_id,
    id2label=id_to_label,
    ignore_mismatched_sizes=True
)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Thiết lập cấu hình Lora

In [13]:
from peft import get_peft_model, LoraConfig, TaskType

print("Cấu hình LoRA...")
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS,
    inference_mode=False,
    r=32,                          # Rank của ma trận thích ứng
    lora_alpha=64,                 # Hệ số ảnh hưởng của LoRA
    lora_dropout=0.1,              # Tỷ lệ dropout
    target_modules=["query", "key", "value"],  # Các module cần áp dụng LoRA
    bias="none",
)

Cấu hình LoRA...


### Áp dụng Lora lên mô hình

In [14]:
# 9. Áp dụng LoRA lên mô hình
print("Áp dụng LoRA cho mô hình...")
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # In ra số lượng tham số cần huấn luyện
model = model.to(device)

Áp dụng LoRA cho mô hình...
trainable params: 1,771,779 || all params: 136,181,766 || trainable%: 1.3010


In [15]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
)

# Cấu hình tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results_lora",
    learning_rate=1e-4,           # Tăng nhẹ learning rate so với fine-tuning thông thường
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,           # Tăng số epoch khi dùng LoRA
    weight_decay=0.01,
    eval_strategy="epoch",        # Đã sửa từ evaluation_strategy thành eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    push_to_hub=False,
    report_to="none",             # Không báo cáo metrics ra ngoài

)

# Tạo trainer
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator = data_collator
)

# Huấn luyện
trainer.train()

/tmp/ipykernel_19/2290487994.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForTokenClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.564181
2,No log,0.477615
3,No log,0.447606
4,No log,0.435037
5,0.539300,0.430904


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=615, training_loss=0.5184958915400312, metrics={'train_runtime': 430.5866, 'train_samples_per_second': 45.508, 'train_steps_per_second': 1.428, 'total_flos': 2575742258558448.0, 'train_loss': 0.5184958915400312, 'epoch': 5.0})

In [16]:
print(model)

PeftModelForTokenClassification(
  (base_model): LoraModel(
    (model): RobertaForTokenClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(64001, 768, padding_idx=1)
          (position_embeddings): Embedding(258, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDic

In [17]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

## 4. Tạo hàm dự đoán nhãn trên 1 câu

In [18]:
# Lặp qua từng văn bản trong cột 'text' của DataFrame
def predict_ner(text):

    # Tokenize văn bản
    new_tokens = tokenizer(text, truncation=True, padding=True, return_tensors="pt")

    # Di chuyển dữ liệu vào thiết bị (GPU nếu có)
    new_tokens = {key: value.to(device) for key, value in new_tokens.items()}

    # Dự đoán
    outputs = model(**new_tokens)
    predictions = outputs.logits.argmax(dim=-1)

    # Chuyển đổi id của nhãn thành nhãn tương ứng
    id_to_label = model.config.id2label  # lấy từ mô hình (phải đảm bảo model có cấu hình này)
    predicted_labels = [id_to_label[label_id] for label_id in predictions[0].tolist()]

    # Chuyển đổi input_ids thành tokens
    tokens = tokenizer.convert_ids_to_tokens(new_tokens["input_ids"][0])
    tokens = [token.replace("@@", "") for token in tokens]
    return tokens, predicted_labels

## 5. Thực hiện ghép các token

In [19]:
def extract_entities(tokens, labels):
    entities = []
    curent_entity = []

    for token, label in zip(tokens, labels):
        if label == "B-ENTITY":
            # Nếu gặp B-ENTITY, bắt đầu một thực thể mới
            if curent_entity:
                entities.append(" ".join(curent_entity).replace("\u2581", " ").strip())
            curent_entity = [token]
        elif label == "I-ENTITY":
            # Nếu gặp I-ENTITY, thêm vào thực thể hiện tại
            if curent_entity:
                curent_entity.append(token)
        else:
            # Nếu gặp nhãn O, kết thúc thực thể hiện tại (nếu có)
            if curent_entity:
                entities.append(" ".join(curent_entity).replace("\u2581", " ").strip())
                curent_entity = []
    # Lưu thực thể cuối cùng (nếu có)
    if curent_entity:
        entities.append(" ".join(curent_entity).replace("\u2581", " ").strip())
    return set(entities)

In [20]:
# Thực thi
text =" Ngày 18/6 tới, dự án trọng điểm quốc gia: Vành đai 3 TP.HCM - sẽ chính thức được khởi công. Đây được xem là dự án đạt kỷ lục, hình mẫu về tiến độ giải phóng ... youtube.com"
tokens, predicted_labels = predict_ner(text)

entities = extract_entities(tokens,predicted_labels)
print("Extracted Entities:",entities)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Extracted Entities: {'dự án trọng điểm quốc gia : Vành đai 3 TP. HCM - sẽ chính thức được khởi cô'}


## 6. Tạo lại text từ tập test_dataset

In [21]:
Tokens_test = []
for item in test_dataset:
    Tokens_test.append(tokenizer.convert_ids_to_tokens(item["input_ids"]))

def reconstruct_bpe_sentence(tokens):
    sentence = ""
    skip_tokens = {"<s>", "</s>", "<unk>", "<pad>"}

    for token in tokens:
        if token in skip_tokens:
            continue

        if token.endswith("@@"):
            sentence += token[:-2]  # bỏ @@ và không thêm khoảng trắng
        else:
            sentence += token + " "  # token hoàn chỉnh → thêm khoảng trắng

    # Làm sạch dư thừa
    return sentence.strip()

texts = [reconstruct_bpe_sentence(tokens) for tokens in Tokens_test]

### Kết quả dự đoán 980 câu trong tập test

In [22]:
results = []

for text in texts:
    tokens, predicted_labels = predict_ner(text)
    entities = extract_entities(tokens,predicted_labels)
    results.append(entities)
print(results[0])

{'A ĐẶ C L AR O SEE LO ẠI hộp', 'S', 'bánh Flan', 'Pha chế', 'Sữa đặc La Ros ée', 'sinh tố', 'Ữ', 'Cà phê sữ a', 'sữa chua'}


## Đánh giá model trên tập test

In [23]:
def evaluate_model(model, test_dataset, tokenizer, device):
    from sklearn.metrics import classification_report
    
    model.eval()
    all_true_labels = []
    all_predicted_labels = []
    
    # Chuyển test_dataset thành dạng dễ xử lý
    test_input_ids = [torch.tensor(x) for x in test_dataset["input_ids"]]
    test_labels = test_dataset["labels"]
    
    for i, input_ids in enumerate(test_input_ids):
        # Chuyển thành batch
        input_ids = input_ids.unsqueeze(0).to(device)
        
        # Dự đoán
        with torch.no_grad():
            outputs = model(input_ids=input_ids)
        
        predictions = outputs.logits.argmax(dim=-1)[0].cpu().numpy()
        true_labels = test_labels[i]
        
        # Lọc bỏ các nhãn là -100 (padding tokens)
        valid_indices = [j for j, label in enumerate(true_labels) if label != -100]
        true_labels = [true_labels[j] for j in valid_indices]
        predictions = [predictions[j] for j in valid_indices]
        
        all_true_labels.extend(true_labels)
        all_predicted_labels.extend(predictions)
    
    # Chuyển các nhãn dạng số thành dạng chữ để dễ đọc trong báo cáo
    true_labels_str = [id2label[label] for label in all_true_labels]
    pred_labels_str = [id2label[label] for label in all_predicted_labels]
    
    # In báo cáo đánh giá
    print(classification_report(true_labels_str, pred_labels_str))

print("Đánh giá mô hình trên tập test...")
evaluate_model(model, test_dataset, tokenizer, device)

Đánh giá mô hình trên tập test...
              precision    recall  f1-score   support

    B-ENTITY       0.71      0.62      0.66      5737
    I-ENTITY       0.72      0.72      0.72     18880
           O       0.88      0.89      0.89     48880

    accuracy                           0.83     73497
   macro avg       0.77      0.75      0.76     73497
weighted avg       0.83      0.83      0.83     73497

